## Setup e Import

In [1]:
import logging
from icecream import ic
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Problem import Problem


In [2]:
P = Problem(20, density = 0.2, alpha = 1, beta = 1)
# P.plot()

In [3]:
from utils import *

baseline_path = get_baseline_path(P) 
# baseline_path
epsilon = 1e-7
print(f"My custom cost function verified: {abs(compute_cost(P, baseline_path)-P.baseline()) < epsilon}")
print(f"Test shortest path computed: {compute_path_heavy_cities(P, 12, 10)}")

My custom cost function verified: True
Test shortest path computed: [(0, 0), (12, 10), (0, 0)]


## Test della soluzione greedy

In [4]:
import importlib
import pandas as pd
from Problem import Problem
from utils import compute_cost

# Ricarica la soluzione
import s339425
importlib.reload(s339425)
from s339425 import solution

# TUTTE le configurazioni di test
n_cities = [10, 50, 100]  # Escludo 1000 per velocità
alpha_values = [0.0, 1.0, 2.0, 4.0]
beta_values = [0.5, 1, 2, 4]
density_values = [0.2, 0.5, 1.0]
seed = 42  # Fisso per riproducibilità

configs = []
for size in n_cities:
    for density in density_values:
        for alpha in alpha_values:
            for beta in beta_values:
                configs.append([size, density, alpha, beta])

results = []
for size, density, alpha, beta in configs:
    print(f"\nTesting: Size={size}, Density={density}, Alpha={alpha}, Beta={beta}")
    test_p = Problem(size, density=density, alpha=alpha, beta=beta, seed=seed)
    
    baseline_cost = test_p.baseline()
    my_path = solution(test_p)
    my_cost = compute_cost(test_p, my_path)
    improvement = (baseline_cost - my_cost) / baseline_cost * 100
    
    print(f"  Baseline: {baseline_cost:.2f}, My: {my_cost:.2f}, Improvement: {improvement:.2f}%")
    
    results.append({
        'Size': size,
        'Density': density,
        'Alpha': alpha,
        'Beta': beta,
        'Baseline': f"{baseline_cost:.2f}",
        'Solution': f"{my_cost:.2f}",
        'Improv%': f"{improvement:.2f}%"
    })

df = pd.DataFrame(results)
print("\n" + "="*80)
print(df.to_string(index=False))
print("="*80)
df.to_csv('test_results.csv', index=False)  # Salva risultati



Testing: Size=10, Density=0.2, Alpha=0.0, Beta=0.5
  Baseline: 16.76, My: 8.83, Improvement: 47.30%

Testing: Size=10, Density=0.2, Alpha=0.0, Beta=1
  Baseline: 16.76, My: 8.83, Improvement: 47.30%

Testing: Size=10, Density=0.2, Alpha=0.0, Beta=2
  Baseline: 16.76, My: 10.26, Improvement: 38.78%

Testing: Size=10, Density=0.2, Alpha=0.0, Beta=4
  Baseline: 16.76, My: 10.26, Improvement: 38.78%

Testing: Size=10, Density=0.2, Alpha=1.0, Beta=0.5
  Baseline: 301.47, My: 186.42, Improvement: 38.16%

Testing: Size=10, Density=0.2, Alpha=1.0, Beta=1
  Baseline: 4896.29, My: 4888.50, Improvement: 0.16%

Testing: Size=10, Density=0.2, Alpha=1.0, Beta=2
  Baseline: 1835859.98, My: 3231874.25, Improvement: -76.04%

Testing: Size=10, Density=0.2, Alpha=1.0, Beta=4
  Baseline: 413136080204.09, My: 1960123128570.33, Improvement: -374.45%

Testing: Size=10, Density=0.2, Alpha=2.0, Beta=0.5
  Baseline: 419.40, My: 258.79, Improvement: 38.29%

Testing: Size=10, Density=0.2, Alpha=2.0, Beta=1
  Bas